In [1]:
import numpy as np
import pandas as pd
import pymc3 as pm
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import theano
import theano.tensor as tt
import scipy
from os import listdir
from os.path import isfile, join
import xlrd
%matplotlib inline

In [2]:
path = '../RetweetDataAOAS/retweet_data/'
root_tweet_names = [f for f in listdir(path) if isfile(join(path, f))]
num_root_tweets = len(root_tweet_names)

In [3]:
tweet_name_to_index = {}
for i in range(num_root_tweets):
    tweet_name_to_index[root_tweet_names[i]] = i

In [4]:
def format_partition_file_name(name):
    root = name.split('.')
    items = root[0].split('_')
    items[-2], items[-1] = items[-1], items[-2]
    return ".".join(["_".join(items), root[-1]])

In [5]:
partition_path = '../Partition_1.xlsx'
partition = pd.read_excel(partition_path)
partition_assignments = {}
for index, row in partition.iterrows():
    training_file_name = format_partition_file_name(row['Training'])
    prediction_file_name = format_partition_file_name(row['Prediction'])
    partition_assignments[tweet_name_to_index[training_file_name]] = True
    partition_assignments[tweet_name_to_index[prediction_file_name]] = False

In [6]:
# Produces a dictionary of dataframes for each tweetfile, with initial 
# preprocessing
observation_probability = .25
fields = ['RetweetCount', 'UserId', 'ScreenName', 'FollowerCount', 
          'DistanceFromRoot','Time', 'ParentScreenName', 'Text']
tweet_dfs = []
for i in range(num_root_tweets):
    tweet_df = pd.read_csv(path+root_tweet_names[i], sep="\t", header=None, 
                         quoting=csv.QUOTE_NONE, names=fields, encoding = "ISO-8859-1")
    if not partition_assignments[i]:
        tweet_df = tweet_df.head(int(observation_probability * tweet_df.shape[0]))
    tweet_df['Time'] = pd.to_datetime(tweet_df['Time'])

    screen_name_index = {}
    for index, row in tweet_df.iterrows():
        screen_name_index[row['ScreenName']] = index
    tweet_df['ParentDfIndex'] = tweet_df['ParentScreenName'].map(screen_name_index)
    tweet_df.fillna(0)
    tweet_dfs.append(tweet_df)

In [ ]:
def generate_time_deltas(tweet_df):
    t_x = tweet_df.tail(1)['Time']
    time_deltas = []
    for index, row in tweet_df.iterrows():
        time_delta = t_x - row['Time']
        time_deltas.append(time_delta.seconds)
    return time_deltas


In [7]:
# Returns a dictionary of reaction times S_j^x keyed by user id
def generate_reaction_times(tweet_df):
    reaction_times = []
    for index, row in tweet_df.iterrows():
        if index > 0:
            reaction_time = row['Time'] - tweet_df.at[row['ParentDfIndex'],
                                                      'Time']
            reaction_times.append(reaction_time)
    return reaction_times

In [8]:
# Returns a dictionary of M_j^x keyed by user id
def generate_number_of_follower_who_retweet(tweet_df):
    number_of_follower_who_retweet = {}
    for index, row in tweet_df.iterrows():
        if row['UserId'] not in number_of_follower_who_retweet:
            number_of_follower_who_retweet[row['UserId']] = 0
        parent_user_id = tweet_df.at[row['ParentDfIndex'], 'UserId']
        number_of_follower_who_retweet[parent_user_id] += 1
    return number_of_follower_who_retweet

In [9]:
def generate_graph_info(tweet_df, rt_dic):
    depth = []
    num_followers = []
    num_retweets = []
    for index, row in tweet_df.iterrows():
        if row['UserId'] != 'None' and row['DistanceFromRoot'] < 1:
            depth.append(row['DistanceFromRoot'])
            if row['FollowerCount'] == 'None':
                num_followers.append(0)
            else:
                num_followers.append(int(row['FollowerCount']))
            if row['UserId'] in rt_dic:
                num_retweets.append(rt_dic[row['UserId']])
            else:
                num_reweets.append(0)
    return depth, num_followers, num_retweets

In [19]:
hyperparams = {}
log_s_j_x = []
# These three are parallel arrays for each retweet 
# (depth[i], num_followers[i], num_retweets[i] all refer to the same retweet)
depth = []
num_followers = []
num_retweets = []

# Prediction arrays
t = []
T = []
S_x = []
m_t = []
for i in range(len(root_tweet_names)):
    if partition_assignment[i]:
        s_j_x = generate_reaction_times(tweet_dfs[i])
        log_s_j_x.append([np.log(i.seconds) for i in s_j_x])
        M_j_dic = generate_number_of_follower_who_retweet(tweet_dfs[i])
        d_x, M_j_x, m_j_x = generate_graph_info(tweet_dfs[i], M_j_dic)
        depth.extend(d_x)
        num_followers.extend(M_j_x)
        num_retweets.extend(m_j_x)
    else:
        t.append(tweet_dfs[i].tail(1)['Time'])
        T.append(generate_time_deltas(tweet_dfs[i]))
        S_x.append(generate_reaction_times(tweet_dfs[i]))
        M_j_dic = generate_number_of_follower_who_retweet(tweet_dfs[i])
        d_x, M_j_x, m_j_x = generate_graph_info(tweet_dfs[i], M_j_dic)
        m_t.append(m_j_x)
depth = np.array(depth)
num_followers = np.array(num_followers)
num_retweets = np.array(num_retweets)

In [ ]:
# Training on the time-related hyperparameters
with pm.Model() as time_model:
    # global model parameters
    alpha = pm.Normal('alpha', mu=0, sd=100)
    sigma_squared_delta = pm.InverseGamma('sigma_squared_delta', alpha=2, beta=2)
    log_a_tau = pm.Normal('log_a_tau', mu=0, sd=10)
    b_tau = pm.Gamma('b_tau', alpha=1, beta=.002)
    
    # log-normal model for reaction times, nonrecursive...
    a_tau = pm.Deterministic('a_tau', pm.math.exp(log_a_tau))
    for i in range(num_root_tweets):
        t_x = pm.InverseGamma('tau_x_squared_' + str(i), alpha=a_tau, beta=b_tau)
        a_x = pm.Normal('alpha_x_' + str(i), mu=alpha, tau=1/sigma_squared_delta)        
        l_x = pm.Normal('log_s_j_x_' + str(i), mu=a_x, sd=t_x**0.5, observed=log_s_j_x[i])
        
# Run and fit our model
with time_model:
    trace = pm.sample(1000, tune=2000, cores=4)
    time_params = ['alpha', 'sigma_squared_delta', 'a_tau', 'b_tau']
    # Extract the hyperparameters
    for param in time_params:
        hyperparams[param] = np.mean(trace[param])

In [21]:
# Training on the graph related hyperparameters
with pm.Model() as graph_model:
    sigma_squared_b = pm.InverseGamma('sigma_squared_b', alpha=0.5, beta=0.5, testval=10000)
    beta_0 = pm.Normal('beta_0', mu=0, tau=1/10000, testval=1.99)
    beta_F = pm.Normal('beta_F', mu=0, tau=1/10000, testval=-0.79)
    beta_d = pm.Normal('beta_d', mu=0, tau=1/10000)
    
    u_j = beta_0 + beta_F * pm.math.log(num_followers+1) + beta_d * pm.math.log(depth+1)
    logit_b_j = pm.Normal('logit_b_j', mu=u_j, tau=1/sigma_squared_b, shape=len(depth))
    b_j = pm.math.invlogit(logit_b_j)
    M_j = pm.Binomial('retweet_count M_j', n=num_followers, p=b_j, observed=num_retweets)
    
    
# Run and fit our model
with graph_model:
    trace = pm.sample(1000, tune=1000, cores=4)
    graph_params = ['sigma_squared_b', 'beta_0', 'beta_F', 'beta_d']
    # Extract the hyperparameters
    for param in graph_params:
        hyperparams[param] = np.mean(trace[param])

In [ ]:
# graph all of the important distributions for time hyper-paramers
with time_model:
    imp_dists = ['alpha', 'sigma_squared_delta', 'a_tau', 'b_tau']
    fig, axs = plt.subplots(2,2, figsize = (8,8))
    for i in range(len(imp_dists)):
        ax1 = axs[int(i/2)][i%2]
        ax2 = ax1.twinx()
        ax2.set(ylim=(-.25, .25))
        var = imp_dists[i]
        sns.distplot(trace[var], ax=ax1).set_title(var);
        sns.boxplot(trace[var], ax=ax2, notch=True)
        print(var + ": " + str(np.mean(trace[var])) + "(" + str(np.std(trace[var])) + ")")


In [23]:
# graph all of the important distributions for graph hyper-paramers
with graph_model:
    imp_dists = ['sigma_squared_b', 'beta_0', 'beta_F', 'beta_d']
#    fig, axs = plt.subplots(2,2, figsize = (8,8))
    for i in range(len(imp_dists)):
#         ax1 = axs[int(i/2)][i%2]
#         ax2 = ax1.twinx()
#         ax2.set(ylim=(-.25, .25))
        var = imp_dists[i]
#         sns.distplot(trace[var], ax=ax1).set_title(var);
#         sns.boxplot(trace[var], ax=ax2, notch=True)
        print(var + ": " + str(np.mean(trace[var])) + "(" + str(np.std(trace[var])) + ")")


sigma_squared_b: 0.906456624789(0.188863500713)
beta_0: 2.14994659679(0.830771062302)
beta_F: -0.789875139102(0.0624924455684)
beta_d: -0.225780593314(102.146961865)


In [ ]:
a_x_prediction = []
t_x_prediction = []
# Sample the a_x, t_x latent variables for the prediction tweets
with pm.Model as time_prediction:
    for i in range(len(prediction_tweets)):
        t_x = pm.InverseGamma('tau_x_squared_' + str(i), alpha=hyperparams.a_tau, beta=hyperparams.b_tau)
        a_x = pm.Normal('alpha_x_' + str(i), mu=hyperparams.alpha, tau=1/hyperparams.sigma_squared_delta)        
        l_x = pm.Normal('log_s_j_x_' + str(i), mu=a_x, sd=t_x**0.5, observed=log_s_j_x[i])
        
# Run and fit our model
with time_prediction:
    trace = pm.sample(1000, tune=1000, cores=4)
    for i in range(len(prediction_tweets)):
        a_x_prediction.append(np.mean(trace['alpha_x_' + str(i)]))
        t_x_prediction.append(np.mean(trace['tau_x_squared_' + str(i)]))
        

In [ ]:
class Retweet(pm.Continuous):
    def __init__(a_x, t_x, S_x, m_t, t, T, *args, **kwargs):
        super(Retweet, self).__init__(*args, **kwargs)
        self.a_x = a_x
        self.t_x = t_x
        self.S_x = S_x
        self.m_t = m_t
        self.t = t
        self.T = T
    
    def logp(self, M):
        nCr = lambda x,y: math.factorial(x) / (math.factorial(y) * math.factorial(x - y))
        choose_func = theano.function([a,b] nCr(a,b))
        choose_term = tt.prod(choose_func(M, self.m_t))

        gauss = scipy.stats.norm(self.a_x, self.t_x)
        F = lambda x: gauss.cdf(x)
        f_func = theano.function([a,b], lambda x,y: (1 - F(x))**y)
        f_term = tt.prod(f_func(tt.log(self.t - self.T), M - self.m_t))


        rxn_term = tt.prod(gauss.pdf(self.S_x))

        return math.log(rxn_term * f_term * choose_term)

In [ ]:
with pm.Model() as graph_prediction:
    for i in range(len(prediction_tweets)):
        # TODO: Generate S_x, m_t, t for each prediction tweet
        likelihood = Retweet(a_x = a_x_prediction[i], t_x = t_x_prection[i], S_x = S_x[i], m_t = m_t[i], t = t[i], T = T[i])
        
with graph_prediction:
    trace = pm.sample(1000, tune=1000, cores=4)